In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import torch
from torch.nn import MSELoss
from skorch.regressor import NeuralNetRegressor
from skorch.dataset import ValidSplit
from skorch.callbacks import EarlyStopping
from sklearn import metrics
from sklearn.model_selection import train_test_split
from torch.nn import Linear
from torch.nn import Sigmoid, ReLU
from torch.nn import Module
from sklearn.metrics import r2_score
from scipy.spatial import distance
from scipy.spatial.distance import euclidean
from torch import tensor
from sklearn.metrics import mean_squared_error
from scipy.spatial.distance import cdist
from sklearn.ensemble import RandomForestRegressor
from torch.nn.init import xavier_uniform_
from model import MLP
from model_state import get_model_params, get_input_for_hidden_layers, get_model_params_gradientNorm
from functions_batch1 import get_variance, qbc, normalization, error_reduct_fuc, total_disagrement, grad_norm, averaged_grad_norm, get_avg_grad_norm, training_loss
import random
from skorch.callbacks import EarlyStopping, LRScheduler, Freezer, Unfreezer
from torch.optim.lr_scheduler import CyclicLR, ReduceLROnPlateau
from skorch.regressor import NeuralNetRegressor, NeuralNet
from model_state import features_concat, learning_state_features_concat
import gc
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Hyper-params:
query_number = 25          # The number of the AL Iterations in each Exp
iteration = 20             # Total number of the Exps
batch_size = 10
total_initail_size = 11
initial_size = 5
dimensions = 18   # Total dimension of the data features + model states

seed_rf = np.load(file="..\..\Seeds\seed2.npy")
seed_initial = np.load(file="..\..\Seeds\seed3.npy")

seed_nn1 = np.load(file="..\..\Seeds\seed4.npy")
seed_nn2 = np.load(file="..\..\Seeds\seed4.npy")
seed_nn3 = np.load(file="..\..\Seeds\seed5.npy")
seed_nn4 = np.load(file="..\..\Seeds\seed5.npy")
seed_nn5 = np.load(file="..\..\Seeds\seed6.npy")
seed_nn6 = np.load(file="..\..\Seeds\seed6.npy")

seed_predictor = np.load(file="..\..\Seeds\seed7.npy")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def Committee_Prediction(learner_list, X_test, y_test):
    initial_pred=[]
    for i in learner_list:
        initial_pred.append(r2_score(y_test, i.predict(X_test)))
    initial_pred=np.array(initial_pred)
    
    return initial_pred.mean()

In [ ]:
def multi_argmax(values, n_instances):

    assert n_instances <= values.shape[0], 'n_instances must be less or equal than the size of utility.'

    max_idx = np.argpartition(-values, n_instances-1, axis=0)[:n_instances]
    return max_idx

In [ ]:
def main_function(query_number, iters):
    # For saving results:
    rf_model_training_r2 = []
    rf_model_training_mse = []
    rf_model_testing_r2 = []
    rf_model_testing_mse = []
    
    # Number of member in committee:
    n_members = 3
    learner_list = []
    
    # The model for evaluation:
    rf_model = RandomForestRegressor(random_state=seed_rf[iters], n_estimators=100)
    gradient_norms = np.empty(shape=(0, 0))
    
    # Load the data:
    name1 = "..\..\Datasets\Diabetes\X_train" + str(iters) + ".npy"
    name2 = "..\..\Datasets\Diabetes\X_test" + str(iters) + ".npy"
    name3 = "..\..\Datasets\Diabetes\y_train" + str(iters) + ".npy"
    name4 = "..\..\Datasets\Diabetes\y_test" + str(iters) + ".npy"
    
    X_train = np.load(name1, allow_pickle=True).astype(np.float32)
    X_test = np.load(name2, allow_pickle=True).astype(np.float32)
    y_train = np.load(name3, allow_pickle=True).astype(np.float32).reshape(-1, 1)
    y_test = np.load(name4, allow_pickle=True).astype(np.float32).reshape(-1, 1)
    
    # Dimensionality of data:
    X = X_train.shape[1]
    # The index of the unlabeled pool:
    X_index = np.arange(X_train.shape[0])

    used_data = np.empty(shape=(0, X))
    used_label = np.empty(shape=(0)).reshape(-1, 1)

    X_initial = np.empty(shape=(0,X))
    y_initial = np.empty(shape=(0)).reshape(-1, 1)
    
    # Initial Stage 1:
    np.random.seed(seed_initial[iters])
    idx = np.random.choice(range(len(X_index)), size=initial_size, replace=False)
    train_idx = X_index[idx]

    X_initial = X_train[train_idx]
    y_initial = y_train[train_idx].reshape(-1, 1)

    used_data = np.append(used_data, X_initial, axis=0).astype(np.float32)
    used_label = np.append(used_label, y_initial, axis=0).astype(np.float32).reshape(-1, 1)
    X_index = np.delete(X_index, idx, axis=0)

    used_data = torch.from_numpy(used_data).to(device)
    used_label = torch.from_numpy(used_label).to(device)
    for member_idx in range(n_members):
        if member_idx == 0:
            np.random.seed(seed_nn1[iters])
            torch.manual_seed(seed_nn2[iters])
        if member_idx == 1:
            np.random.seed(seed_nn3[iters])
            torch.manual_seed(seed_nn4[iters])
        if member_idx == 2:
            np.random.seed(seed_nn5[iters])
            torch.manual_seed(seed_nn6[iters])
            
        regressor = NeuralNetRegressor(MLP(X),
                                   criterion=MSELoss(),
                                   optimizer=torch.optim.Adam,
                                   verbose=0,
                                   max_epochs=30,
                                   lr=0.001,
                                   # Used for the batch AL
                                   callbacks=[EarlyStopping(patience=5), ('lr_scheduler', LRScheduler(policy=ReduceLROnPlateau))],
                                   train_split=ValidSplit(cv=5),
                                   warm_start=True,
                                   device='cuda',
                                   batch_size = 200
                                   )
        regressor.fit(used_data, used_label)
        learner_list.append(regressor)
    
    print("NN Test R2 with 5 samples", Committee_Prediction(learner_list, X_test, y_test))

    # Number of LAL features:
    learning_state_features = np.empty(shape=(0, dimensions))
    loss_reduction_target = np.empty(shape=(0)).reshape(-1, 1)

    # other random samples:
    rest_initial_X = np.empty(shape=(0, X))
    rest_initial_y = np.empty(shape=(0)).reshape(-1, 1)

    # LAL Initialization: Stage 2
    predictor = RandomForestRegressor(n_estimators=1000, random_state=seed_predictor[iters])
    np.random.seed(seed_initial[iters])
    idx = np.random.choice(range(len(X_index)), size=total_initail_size-initial_size, replace=False)
    train_idx = X_index[idx]
    
    rest_initial_X = np.append(rest_initial_X, X_train[train_idx], axis=0).astype(np.float32)
    rest_initial_y = np.append(rest_initial_y, y_train[train_idx], axis=0).astype(np.float32).reshape(-1, 1)

    X_index = np.delete(X_index, idx, axis=0)

    for i in range(rest_initial_X.shape[0]):

        single_X = rest_initial_X[i],
        single_y = rest_initial_y[i].reshape(1, -1)
        single_X = single_X[0].reshape(1, -1)
        
        single_X = torch.from_numpy(single_X).to(device)
        single_y = torch.from_numpy(single_y).to(device)

        # Model params
        model_params = get_model_params_gradientNorm(learner_list)

        # updated data
        used_data = torch.cat((used_data, single_X), 0)
        used_label = torch.cat((used_label, single_y), 0)

        # Retrain the model:
        for l in learner_list:
            l.fit(X=used_data,y=used_label)
            
        single_X = single_X.cpu().numpy()
        single_y = single_y.cpu().numpy()
            
        index = train_idx[i]

        # The training sample:
        data_params = X_train[index]
      
        LALfeatures = features_concat(model_params, data_params)

        loss_reduction = get_avg_grad_norm(learner_list, single_X, single_y)
        
        learning_state_features = learning_state_features_concat(learning_state_features, LALfeatures)
        loss_reduction_target = np.append(loss_reduction_target, np.array([loss_reduction]).reshape(-1, 1), axis=0).astype(np.float32).reshape(-1, 1)

        
    used_data = used_data.cpu().numpy()
    used_label = used_label.cpu().numpy()

    # Finished the Initialization Stages:
    # RF Regressor for evaluation:
    rf_model.fit(used_data, used_label.ravel())
    # Training Scores:
    rf_training_r2 = r2_score(used_label, rf_model.predict(used_data))
    rf_training_mse = mean_squared_error(used_label, rf_model.predict(used_data))
    rf_model_training_r2.append(rf_training_r2)
    rf_model_training_mse.append(rf_training_mse)
    
    # Test Scores:
    rf_model_r2 = r2_score(y_test, rf_model.predict(X_test))
    rf_model_mse = mean_squared_error(y_test, rf_model.predict(X_test))
    rf_model_testing_r2.append(rf_model_r2)
    rf_model_testing_mse.append(rf_model_mse)
    
    print("After Initialization RF R2:", rf_model_r2)
    print("NN Test R2 after Initialization", Committee_Prediction(learner_list, X_test, y_test))

    print(np.unique(used_data, axis=0).shape)
    for idx in range(query_number):
        np.random.seed(None)
        print('Query no. %d' % (idx+1))

        Error_reduction_list, training_score = error_reduct_fuc(X_train, X_index, learner_list, learning_state_features, loss_reduction_target, X_index.shape[0], predictor)

        # idx = multi_argmax(np.array(Error_reduction_list), n_instances=batch_size)
        # Inversed Experiments:
        if any(x < 0 for x in Error_reduction_list):
            print("The NumPy Array has at least one negative value")
        idx = multi_argmax(-1*np.array(Error_reduction_list), n_instances=batch_size)
        ####### 
        
        X_train_indices = X_index[idx]
        
        # Update
        X_index = np.delete(X_index, idx, axis=0)

        for X_train_index in X_train_indices:

            new_X = X_train[X_train_index].reshape(1, -1)
            new_y = y_train[X_train_index].reshape(1, -1)
            
            used_data = np.append(used_data, new_X, axis=0).astype(np.float32)
            used_label = np.append(used_label, new_y, axis=0).astype(np.float32).reshape(-1, 1)
            
            used_data = torch.from_numpy(used_data).to(device)
            used_label = torch.from_numpy(used_label).to(device)

            # Model params
            model_params = get_model_params_gradientNorm(learner_list)
            data_params = X_train[X_train_index]
            LALfeatures = features_concat(model_params, data_params)
            
            for l in learner_list:
                l.fit(used_data, used_label)
            
            used_data = used_data.cpu().numpy()
            used_label = used_label.cpu().numpy()

            loss_reduction = get_avg_grad_norm(learner_list, new_X, new_y)  
            gradient_norms = np.append(gradient_norms, loss_reduction)
            
            learning_state_features = learning_state_features_concat(learning_state_features, LALfeatures)
            loss_reduction_target = np.append(loss_reduction_target, np.array([loss_reduction]).reshape(-1, 1), axis=0).astype(np.float32).reshape(-1, 1)
        

        # RF Evaluation
        rf_model.fit(used_data, used_label.ravel())
        
        # Training Evaluation:
        rf_training_r2 = r2_score(used_label, rf_model.predict(used_data))
        rf_training_mse = mean_squared_error(used_label, rf_model.predict(used_data))
        rf_model_training_r2.append(rf_training_r2)
        rf_model_training_mse.append(rf_training_mse)
        
        # Test Evaluation:
        rf_model_r2 = r2_score(y_test, rf_model.predict(X_test))
        rf_model_mse = mean_squared_error(y_test, rf_model.predict(X_test))
        rf_model_testing_r2.append(rf_model_r2)
        rf_model_testing_mse.append(rf_model_mse)

        print(np.unique(used_data, axis=0).shape)
        print("RF R2:", rf_model_r2)
        print("Remaining:", X_index.shape[0])
        # print('NN R2', Committee_Prediction(learner_list, X_test, y_test))

    # RF
    rf_model_testing_r2 = np.array(rf_model_testing_r2)
    rf_model_testing_mse = np.array(rf_model_testing_mse)
    rf_model_training_r2 = np.array(rf_model_training_r2)
    rf_model_training_mse = np.array(rf_model_training_mse)
    
    # Used_data and Unsed_label:
    np.save(file="..\..\Results\Res_Diabetes\OMAL_inverse\Summary\\used_data" + str(iters) + ".npy", arr=used_data)
    np.save(file="..\..\Results\Res_Diabetes\OMAL_inverse\Summary\\used_labels" + str(iters) + ".npy", arr=used_label)

    np.save(file="..\..\Results\Res_Diabetes\OMAL_inverse\Summary\\testing_rf_r2_" + str(iters) + ".npy", arr=rf_model_testing_r2)
    np.save(file="..\..\Results\Res_Diabetes\OMAL_inverse\Summary\\testing_rf_mse_" + str(iters) + ".npy", arr=rf_model_testing_mse)

In [ ]:
for i in range(iteration):
    print("The Iteration is ", i)
    main_function(query_number, i)